In [1]:
#packages
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,.2f}'.format

#some basic functions
def print_shape(dataframe,name):
    #prints dataframe shape and name, name must be in quotes
    # example print(dataframe,'my dataframe')
    print(name+" has: {} rows and {} columns".format(dataframe.shape[0],dataframe.shape[1]))
    

#some less basic functions
def movie_series_parser(x):
    #counts how many semicolons occour, if 1 or zero, assume a movie, if two or more assume a series
    #determines if text is a movie or series by looking to see if the word "Season" or "Part" after a semicolon
    #if it does, a new column will be created called "Season" in the dataframe and that text will be entered there
    #the rest of the function will then delete everthing after the word inclusive of ": Season" or ": Part"
    Movie_or_series = ""
    Season = ""
    Title_of_production = ""
    
    if x.count(":") <2:
        Movie_or_series = 'Movie'
        Title_of_production = x
        return Movie_or_series, Title_of_production, Season
    else:
        Movie_or_series = 'Series'
        try:
            Season_search = re.search(r"(: Season \d)|(: Part \d)",x)
            Season = Season_search.group(1)[2:]
        except:
            Season = 'None'        
        try:    
            Title_search = re.sub(r'(: Season .*)|(: Part .*)','',x)
            Title_of_production = Title_search
        except:
            Title_of_production = x
   
        
        return Movie_or_series, Title_of_production, Season
    
#import data
viewing_activity_df = pd.read_csv('data/ViewingActivity.csv',parse_dates = ['Start Time'])
billing_history_df = pd.read_csv('data/BillingHistory.csv',parse_dates=['Transaction Date'])

#clean up viewing history data
#latest view is duplicate data and supplemental video are netflix ads
#convert duration to a floating numbers
viewing_activity_df = viewing_activity_df[viewing_activity_df['Latest Bookmark']!='Not latest view']
viewing_activity_df = viewing_activity_df[viewing_activity_df['Supplemental Video Type'].isnull()].reset_index(drop=True)
viewing_activity_df['Duration_floating_hours'] = pd.to_timedelta(viewing_activity_df['Duration'],unit='hours').dt.total_seconds()/3600
#add new columns, year, month, hour, weekday, add if movie or series, if series which season
viewing_activity_df['Year'] = viewing_activity_df['Start Time'].dt.year
viewing_activity_df['Month'] = viewing_activity_df['Start Time'].dt.month
viewing_activity_df['Hour'] = viewing_activity_df['Start Time'].dt.hour
viewing_activity_df['Weekday'] = viewing_activity_df['Start Time'].dt.dayofweek
weekday_dict = {0:'Sunday',1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday'}
viewing_activity_df["Weekday"].replace(weekday_dict, inplace=True)
#apply movie parser function to determine if this is a movie, series and what season if series
viewing_activity_df['Movie_or_series'], viewing_activity_df['Title_of_production'], viewing_activity_df['Season'] = zip(*viewing_activity_df['Title'].apply(movie_series_parser))
viewing_activity_df.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,Duration_floating_hours,Year,Month,Hour,Weekday,Movie_or_series,Title_of_production,Season
0,Brad,2020-07-11 02:51:12,01:55:38,NaN,The Silence of the Lambs,NaN,Vizio MG244 FXCN SX7 CAST Smart TV,01:55:38,01:55:38,US (United States),1.93,2020,7,2,Friday,Movie,The Silence of the Lambs,
1,Brad,2020-07-11 00:38:45,00:59:12,NaN,The Devil's Advocate,NaN,Vizio MG244 FXCN SX7 CAST Smart TV,02:18:09,02:18:09,US (United States),0.99,2020,7,0,Friday,Movie,The Devil's Advocate,
2,Brad,2020-07-10 02:47:20,00:05:43,NaN,Hannibal: Season 3: Contorno (Episode 5),NaN,Vizio MG244 FXCN SX7 CAST Smart TV,00:35:09,00:35:09,US (United States),0.10,2020,7,2,Thursday,Series,Hannibal,Season 3
3,Brad,2020-07-09 02:33:00,00:43:31,NaN,Abstract: The Art of Design: Season 2: Jonatha...,NaN,Vizio MG244 FXCN SX7 CAST Smart TV,00:43:31,00:43:31,US (United States),0.73,2020,7,2,Wednesday,Series,Abstract: The Art of Design,Season 2
4,Brad,2020-07-08 23:21:31,00:42:23,NaN,Hannibal: Season 3: Aperitivo (Episode 4),NaN,Vizio MG244 FXCN SX7 CAST Smart TV,00:42:43,00:42:43,US (United States),0.71,2020,7,23,Tuesday,Series,Hannibal,Season 3


In [6]:
viewing_activity_df[(viewing_activity_df['Start Time'] >='2013-02-01')&(viewing_activity_df['Start Time'] <='2013-03-28')]

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,Duration_floating_hours,Year,Month,Hour,Weekday,Movie_or_series,Title_of_production,Season
5222,Brad,2013-03-23 23:23:11,00:00:11,NaN,Top Gear: Series 18: Episode 5,NaN,Microsoft Xbox 360,00:52:05,00:52:05,US (United States),0.00,2013,3,23,Friday,Series,Top Gear: Series 18: Episode 5,None
5223,Brad,2013-03-22 22:30:31,00:23:19,NaN,eXistenZ,NaN,Microsoft Xbox 360,01:34:31,01:34:31,US (United States),0.39,2013,3,22,Thursday,Movie,eXistenZ,
5224,Brad,2013-03-17 10:13:20,02:02:38,NaN,Runaway Bride,NaN,Samsung 2011 Genoa-S Smart TV,01:52:49,01:52:49,US (United States),2.04,2013,3,10,Saturday,Movie,Runaway Bride,
5225,Brad,2013-03-17 01:17:27,00:58:27,NaN,The Cider House Rules,NaN,Samsung 2011 Genoa-S Smart TV,02:01:31,02:01:31,US (United States),0.97,2013,3,1,Saturday,Movie,The Cider House Rules,
5226,Brad,2013-03-16 19:15:18,02:01:32,NaN,Mirror Mirror,NaN,Samsung 2011 Genoa-S Smart TV,01:40:24,01:40:24,US (United States),2.03,2013,3,19,Friday,Movie,Mirror Mirror,
5227,Brad,2013-03-15 22:06:01,00:51:55,NaN,The Hunter,NaN,Microsoft Xbox 360,00:51:59,00:51:59,US (United States),0.87,2013,3,22,Thursday,Movie,The Hunter,
5228,Brad,2013-03-03 15:51:58,00:08:30,NaN,Cashback,NaN,Mac,00:29:28,00:29:28,US (United States),0.14,2013,3,15,Saturday,Movie,Cashback,
5229,Brad,2013-03-02 14:31:09,00:11:28,NaN,The Secret of My Success,NaN,Samsung 2011 Genoa-S Smart TV,01:49:50,01:49:50,US (United States),0.19,2013,3,14,Friday,Movie,The Secret of My Success,
5230,Brad,2013-03-01 23:28:56,00:25:45,NaN,Iron Sky,NaN,Microsoft Xbox 360,00:25:46,00:25:46,US (United States),0.43,2013,3,23,Thursday,Movie,Iron Sky,
5231,Brad,2013-02-28 01:30:02,00:00:11,NaN,Æon Flux,NaN,Microsoft Xbox 360,00:04:38,00:04:38,US (United States),0.00,2013,2,1,Wednesday,Movie,Æon Flux,
